<a href="https://colab.research.google.com/github/MrBaten/AutoML/blob/main/flipping_augmentation_inception__v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import zipfile
zip_file_path = '/content/drive/MyDrive/Pest.zip'

# Specify the destination directory to extract the contents
destination_path = '/content/'

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(destination_path)

In [ ]:
import keras,os
import numpy as np
import tensorflow as tf
import cv2
from keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Sequential , load_model , Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Dropout, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from tensorflow.keras import applications
from tensorflow.keras import optimizers
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.utils import load_img, img_to_array
from keras.layers import BatchNormalization
from glob import glob
# Step 1: Prepare your dataset
# Assume you have a directory with images and corresponding labels

with tf.device('/gpu:0'):
    pass

train_dir = '/content/pest/train'
valid_dir = '/content/pest/test'

In [ ]:
horizontal_flip = True
vertical_flip = False

batch_size = 256
train_datagen = ImageDataGenerator(rescale=1/255,
                  shear_range=0.3,
                  horizontal_flip = horizontal_flip,
                  vertical_flip = vertical_flip,
                  zoom_range=0.3
                  )
val_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
                train_dir,
                target_size=(224,224),
                batch_size=batch_size,
                color_mode="rgb",
                class_mode="categorical"
                )

val_generator = val_datagen.flow_from_directory(
                valid_dir,
                target_size=(224,224),
                batch_size=batch_size,
                color_mode="rgb",
                class_mode="categorical"
                )

Found 2700 images belonging to 9 classes.
Found 450 images belonging to 9 classes.


In [ ]:
# Initialize empty lists to store the data and labels
train_x = []
train_y = []
test_x = []
test_y = []

# Iterate over the training data generator and collect all batches
for train_batch_x, train_batch_y in train_generator:
    train_x.append(train_batch_x)
    train_y.append(train_batch_y)

    # Check if we have collected all samples
    if len(train_x) * train_generator.batch_size >= len(train_generator.filenames):
        break

# Concatenate the batches to form the complete training dataset
train_x = np.concatenate(train_x, axis=0)
train_y = np.concatenate(train_y, axis=0)

# Iterate over the test data generator and collect all batches
for test_batch_x, test_batch_y in val_generator:
    test_x.append(test_batch_x)
    test_y.append(test_batch_y)

    # Check if we have collected all samples
    if len(test_x) * val_generator.batch_size >= len(val_generator.filenames):
        break

# Concatenate the batches to form the complete test dataset
test_x = np.concatenate(test_x, axis=0)
test_y = np.concatenate(test_y, axis=0)

In [ ]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(2700, 224, 224, 3)
(2700, 9)
(450, 224, 224, 3)
(450, 9)


In [ ]:
IncV3 = InceptionV3(include_top = False, weights = "imagenet",input_shape = (224,224,3))

87910968/87910968 [==============================] - 0s 0us/step


In [ ]:
className = glob(train_dir + "/*")
NumberofClass = len(className)
print("NumberofClass:", NumberofClass)

NumberofClass: 9


In [ ]:
model = Sequential()

model.add(IncV3)

for layer in model.layers:
    layer.trainable = False


model.add(Flatten())

model.add(Dense(units = 2048, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(units = NumberofClass, activation = "softmax"))


In [ ]:
model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

In [ ]:
model.fit(train_x, train_y, batch_size=32, epochs=40, validation_data=(test_x, test_y))

Epoch 1/40
85/85 [==============================] - 27s 151ms/step - loss: 18.8424 - accuracy: 0.6063 - val_loss: 0.7710 - val_accuracy: 0.8667
Epoch 2/40
85/85 [==============================] - 8s 98ms/step - loss: 0.6645 - accuracy: 0.8737 - val_loss: 0.2797 - val_accuracy: 0.9244
Epoch 3/40
85/85 [==============================] - 8s 99ms/step - loss: 0.2968 - accuracy: 0.9119 - val_loss: 0.2441 - val_accuracy: 0.9467
Epoch 4/40
85/85 [==============================] - 9s 101ms/step - loss: 0.1908 - accuracy: 0.9433 - val_loss: 0.2803 - val_accuracy: 0.9311
Epoch 5/40
85/85 [==============================] - 8s 95ms/step - loss: 0.2692 - accuracy: 0.9326 - val_loss: 0.3505 - val_accuracy: 0.9311
Epoch 6/40
85/85 [==============================] - 8s 97ms/step - loss: 0.3855 - accuracy: 0.9163 - val_loss: 0.2164 - val_accuracy: 0.9533
Epoch 7/40
85/85 [==============================] - 9s 100ms/step - loss: 0.3380 - accuracy: 0.9230 - val_loss: 0.4251 - val_accuracy: 0.9133
Epoch 8/

In [ ]:
train_loss, train_accuracy = model.evaluate(train_generator)
test_loss, test_accuracy = model.evaluate(val_generator)

print('Training Accuracy:', train_accuracy)
print('Test Accuracy:', test_accuracy)

2/2 [==============================] - 9s 7s/step - loss: 0.5037 - accuracy: 0.9489
Training Accuracy: 0.9759259223937988
Test Accuracy: 0.948888897895813


In [ ]:
preds = model.evaluate(test_x, test_y)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

15/15 [==============================] - 1s 71ms/step - loss: 0.5036 - accuracy: 0.9489
Loss = 0.5036208629608154
Test Accuracy = 0.948888897895813


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

test_loss, test_acc = model.evaluate(test_x, test_y)
pred_labels = np.argmax(model.predict(test_x), axis=1)

true_labels = np.argmax(test_y, axis=1)

class_accuracies = []
f1_scores = []
avg_precisions = []
avg_recalls = []

for class_label in range(9):
    class_indices = np.where(true_labels == class_label)[0]
    class_pred_labels = pred_labels[class_indices]
    class_true_labels = true_labels[class_indices]

    # Calculate class accuracy
    class_accuracy = np.mean(class_pred_labels == class_true_labels)
    class_accuracies.append(class_accuracy)

    # Calculate F1 score
    class_f1_score = f1_score(class_true_labels, class_pred_labels, pos_label=class_label, average='macro')
    f1_scores.append(class_f1_score)

    # Calculate precision
    class_precision = precision_score(class_true_labels, class_pred_labels, pos_label=class_label, average='macro')
    avg_precisions.append(class_precision)

    # Calculate recall
    class_recall = recall_score(class_true_labels, class_pred_labels, pos_label=class_label, average='macro')
    avg_recalls.append(class_recall)

mean_class_accuracy = np.mean(class_accuracies)
f1_score = np.mean(f1_scores)
mean_precision = np.mean(avg_precisions)
mean_recall = np.mean(avg_recalls)

print("Mean Class Accuracy:", mean_class_accuracy)
print("F1 Score:", f1_score)
print("Mean Average Precision:", mean_precision)
print("Mean Average Recall:", mean_recall)

15/15 [==============================] - 2s 70ms/step
Mean Class Accuracy: 0.9488888888888888
F1 Score: 0.5690039358935096
Mean Average Precision: 0.5777777777777777
Mean Average Recall: 0.5611111111111111


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 0) is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 0) is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 0) is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control

# SGD

In [ ]:
model.compile(optimizer = "SGD", loss = "categorical_crossentropy", metrics = ["accuracy"])

In [ ]:
model.fit(train_x, train_y, batch_size=32, epochs=40, validation_data=(test_x, test_y))

Epoch 1/40
85/85 [==============================] - 15s 121ms/step - loss: 1.2956 - accuracy: 0.9385 - val_loss: 3.7611 - val_accuracy: 0.9244
Epoch 2/40
85/85 [==============================] - 8s 90ms/step - loss: 0.9786 - accuracy: 0.9633 - val_loss: 1.1748 - val_accuracy: 0.9622
Epoch 3/40
85/85 [==============================] - 8s 92ms/step - loss: 0.6421 - accuracy: 0.9689 - val_loss: 1.1365 - val_accuracy: 0.9711
Epoch 4/40
85/85 [==============================] - 8s 91ms/step - loss: 0.3270 - accuracy: 0.9789 - val_loss: 1.3507 - val_accuracy: 0.9644
Epoch 5/40
85/85 [==============================] - 8s 97ms/step - loss: 0.2874 - accuracy: 0.9826 - val_loss: 1.3710 - val_accuracy: 0.9711
Epoch 6/40
85/85 [==============================] - 8s 93ms/step - loss: 0.3129 - accuracy: 0.9833 - val_loss: 1.3398 - val_accuracy: 0.9644
Epoch 7/40
85/85 [==============================] - 8s 92ms/step - loss: 0.3245 - accuracy: 0.9870 - val_loss: 1.2489 - val_accuracy: 0.9667
Epoch 8/40


In [ ]:
train_loss, train_accuracy = model.evaluate(train_generator)
test_loss, test_accuracy = model.evaluate(val_generator)

print('Training Accuracy:', train_accuracy)
print('Test Accuracy:', test_accuracy)

2/2 [==============================] - 2s 690ms/step - loss: 1.2562 - accuracy: 0.9689
Training Accuracy: 0.9855555295944214
Test Accuracy: 0.9688888788223267


In [ ]:
preds = model.evaluate(test_x, test_y)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

15/15 [==============================] - 1s 67ms/step - loss: 1.2562 - accuracy: 0.9689
Loss = 1.2561718225479126
Test Accuracy = 0.9688888788223267


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

test_loss, test_acc = model.evaluate(test_x, test_y)
pred_labels = np.argmax(model.predict(test_x), axis=1)

true_labels = np.argmax(test_y, axis=1)

class_accuracies = []
f1_scores = []
avg_precisions = []
avg_recalls = []

for class_label in range(9):
    class_indices = np.where(true_labels == class_label)[0]
    class_pred_labels = pred_labels[class_indices]
    class_true_labels = true_labels[class_indices]

    # Calculate class accuracy
    class_accuracy = np.mean(class_pred_labels == class_true_labels)
    class_accuracies.append(class_accuracy)

    # Calculate F1 score
    class_f1_score = f1_score(class_true_labels, class_pred_labels, pos_label=class_label, average='macro')
    f1_scores.append(class_f1_score)

    # Calculate precision
    class_precision = precision_score(class_true_labels, class_pred_labels, pos_label=class_label, average='macro')
    avg_precisions.append(class_precision)

    # Calculate recall
    class_recall = recall_score(class_true_labels, class_pred_labels, pos_label=class_label, average='macro')
    avg_recalls.append(class_recall)

mean_class_accuracy = np.mean(class_accuracies)
f1_score = np.mean(f1_scores)
mean_precision = np.mean(avg_precisions)
mean_recall = np.mean(avg_recalls)

print("Mean Class Accuracy:", mean_class_accuracy)
print("F1 Score:", f1_score)
print("Mean Average Precision:", mean_precision)
print("Mean Average Recall:", mean_recall)

15/15 [==============================] - 3s 77ms/step
Mean Class Accuracy: 0.9688888888888888
F1 Score: 0.7077651757249412
Mean Average Precision: 0.7129629629629629
Mean Average Recall: 0.702962962962963


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 0) is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 0) is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 0) is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 2) is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a 

**RMSprop**

In [ ]:
from tensorflow.keras.optimizers import RMSprop

In [ ]:
model.compile(optimizer = "RMSprop", loss = "categorical_crossentropy", metrics = ["accuracy"])

In [ ]:
model.fit(train_x, train_y, batch_size=32, epochs=40, validation_data=(test_x, test_y))

Epoch 1/40
85/85 [==============================] - 15s 115ms/step - loss: 0.3402 - accuracy: 0.9844 - val_loss: 1.3991 - val_accuracy: 0.9689
Epoch 2/40
85/85 [==============================] - 8s 95ms/step - loss: 0.4729 - accuracy: 0.9815 - val_loss: 1.3960 - val_accuracy: 0.9711
Epoch 3/40
85/85 [==============================] - 8s 91ms/step - loss: 0.3475 - accuracy: 0.9867 - val_loss: 1.7108 - val_accuracy: 0.9711
Epoch 4/40
85/85 [==============================] - 8s 93ms/step - loss: 0.5505 - accuracy: 0.9815 - val_loss: 2.2383 - val_accuracy: 0.9667
Epoch 5/40
85/85 [==============================] - 8s 91ms/step - loss: 0.4599 - accuracy: 0.9815 - val_loss: 1.7037 - val_accuracy: 0.9711
Epoch 6/40
85/85 [==============================] - 8s 93ms/step - loss: 0.4364 - accuracy: 0.9852 - val_loss: 1.5243 - val_accuracy: 0.9756
Epoch 7/40
85/85 [==============================] - 8s 96ms/step - loss: 0.4516 - accuracy: 0.9815 - val_loss: 2.0867 - val_accuracy: 0.9644
Epoch 8/40


In [ ]:
train_loss, train_accuracy = model.evaluate(train_generator)
test_loss, test_accuracy = model.evaluate(val_generator)

print('Training Accuracy:', train_accuracy)
print('Test Accuracy:', test_accuracy)

2/2 [==============================] - 2s 629ms/step - loss: 3.4434 - accuracy: 0.9667
Training Accuracy: 0.9877777695655823
Test Accuracy: 0.9666666388511658


In [ ]:
preds = model.evaluate(test_x, test_y)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

15/15 [==============================] - 1s 72ms/step - loss: 3.4433 - accuracy: 0.9667
Loss = 3.4432849884033203
Test Accuracy = 0.9666666388511658


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

test_loss, test_acc = model.evaluate(test_x, test_y)
pred_labels = np.argmax(model.predict(test_x), axis=1)

true_labels = np.argmax(test_y, axis=1)

class_accuracies = []
f1_scores = []
avg_precisions = []
avg_recalls = []

for class_label in range(9):
    class_indices = np.where(true_labels == class_label)[0]
    class_pred_labels = pred_labels[class_indices]
    class_true_labels = true_labels[class_indices]

    # Calculate class accuracy
    class_accuracy = np.mean(class_pred_labels == class_true_labels)
    class_accuracies.append(class_accuracy)

    # Calculate F1 score
    class_f1_score = f1_score(class_true_labels, class_pred_labels, pos_label=class_label, average='macro')
    f1_scores.append(class_f1_score)

    # Calculate precision
    class_precision = precision_score(class_true_labels, class_pred_labels, pos_label=class_label, average='macro')
    avg_precisions.append(class_precision)

    # Calculate recall
    class_recall = recall_score(class_true_labels, class_pred_labels, pos_label=class_label, average='macro')
    avg_recalls.append(class_recall)

mean_class_accuracy = np.mean(class_accuracies)
f1_score = np.mean(f1_scores)
mean_precision = np.mean(avg_precisions)
mean_recall = np.mean(avg_recalls)

print("Mean Class Accuracy:", mean_class_accuracy)
print("F1 Score:", f1_score)
print("Mean Average Precision:", mean_precision)
print("Mean Average Recall:", mean_recall)

15/15 [==============================] - 3s 73ms/step
Mean Class Accuracy: 0.9666666666666666
F1 Score: 0.7719041700177914
Mean Average Precision: 0.7777777777777777
Mean Average Recall: 0.7666666666666667


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 0) is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 0) is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 0) is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 2) is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a 

In [ ]:
import matplotlib.pyplot as plt


def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    #plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()


plot_hist(hist)

NameError: ignored